## Kakuro Matrix

In [ ]:
from gurobipy import *


# 8x8 matrix
kakuro_matrix = [[(None,None),(None,None),(None,None), (None,None), (None,None), (15,None), (16,None), (None, None), (None,None), (None,None), (None,None), (None,None), (None,None)],
                [(None,None),(None,None),(None,None), (None,None), (4,8), 0,0, (4, None), (16,None), (None,None), (None,None), (None,None), (None,None)],
               [(None,None),(None,None),(11,None), (16,22), 0, 0, 0,0,0, (10,None), (None,None), (3,None), (17,None)],
                [(None,None),(None,16),0,0,0,0, (29, 13), 0,0,0, (23,11), 0,0],
               [(None,None),(4,8),0,0, (None,13), 0,0, (None, None), (16,19), 0,0,0,0],
               [(None,4),0,0, (None,None), (None,12), 0,0, (11, 19), 0,0,0, (None,None), (None,None)],
               [(None,8),0,0, (6,None), (11,None), (16,27), 0,0,0,0,0, (30,None), (17,None)],
              [(None,None),(None,None),(None,26), 0,0,0,0,0, (35,None), (None,None), (None,15), 0,0],
               [(None,None),(16,None),(4,11), 0,0,0, (None,11), 0,0, (None,None), (3,17), 0,0],
                 [(None,18),0,0,0,0, (17,None), (4,8), 0,0, (16,11), 0,0, (None,None)],
                 [(None,8),0,0, (None,13), 0,0,0, (4, 21), 0,0,0,0, (None,None)],
                 [(None,None),(None,None),(None,None), (None,None), (None,28), 0,0,0,0,0, (None,None), (None,None), (None,None)],
                 [(None,None),(None,None),(None,None), (None,None), (None,None), (None,None), (None,7), 0,0, (None,None), (None,None), (None,None), (None,None)]
              ]

# 3x3 matrix
# kakuro_matrix = [[(None,None),(16,None),(12,None)],
#                 [(None,17), 0, 0],
#                 [(None,11), 0, 0]
#                 ]

In [ ]:
# Calculating Kakuro Matrix Dimensions
x = len(kakuro_matrix)
y = len(kakuro_matrix[0])

z = range(1, 10) # Each cell can contain values from 1-9
print(x,y,z)

## Kakuro Solver

In [ ]:
m = Model('KakuroBaby')

# Create Decision Variables

var = m.addVars(x,y,z, vtype = GRB.BINARY, name = "VAR")

m.update()

# Set Objective
m.setObjective(0,GRB.MINIMIZE)

# Fill Constraints

#1. every cell that needs to be filled with optimal solution should contain only 1 digit

for i in range(x):
    for j in range(y):
        if kakuro_matrix[i][j]==0:
            m.addConstr(( quicksum(var[i,j,k] for k in z) == 1))

        
# 2nd, 3rd, 4th & 5th across the row and column resp.

for i in range(x):
        for j in range(y):
            
            if kakuro_matrix[i][j] != 0:
                col_sum = 0
                k_col_sum = 0
                value_check_col = kakuro_matrix[i][j][0]
                
                # Take sum value across the column
                
                if value_check_col is not None:
                    i_iter = i + 1
                    
                    for k in z:
                        i_iter_k = i + 1
                        k_col_sum = 0
                        while i_iter_k < x and kakuro_matrix[i_iter_k][j] == 0:
                            k_col_sum += var[i_iter_k,j,k]
                            i_iter_k += 1
                        m.addConstr(k_col_sum <= 1)
                        
                    while i_iter < x and kakuro_matrix[i_iter][j] == 0:
                        k_col_sum = 0
                        for k in z:
                            col_sum += k * var[i_iter,j,k]
                            
                        i_iter += 1
                    m.addConstr(col_sum == value_check_col)
                    
                
                
                # Take sum value across the row
                
                value_check_row = kakuro_matrix[i][j][1]
                row_sum = 0
                k_row_sum = 0

                if value_check_row is not None:
                    
                    for k in z:
                        j_iter = j + 1
                        k_row_sum = 0
                        while j_iter < y and kakuro_matrix[i][j_iter] == 0:
                            k_row_sum += var[i,j_iter,k]
                            j_iter += 1
                        m.addConstr(k_row_sum <= 1)

                    j += 1

                    while j < y and kakuro_matrix[i][j] == 0:
                        for k in z:
                            row_sum += k * var[i,j,k]
                    
            
                        j += 1
                    m.addConstr(row_sum == value_check_row)
                    j -= 1

                                        
m.update()

m.optimize()

### Printing Optimal Solution

In [ ]:
if m.status == GRB.OPTIMAL:
    print("\nOptimal solution:")

    for v in m.getVars():
        if v.x != 0:
            print(f"{v.VarName}: {v.x}")
else:
    print("No optimal solution found.")

### Updating the Kakuro Matrix with the solution

In [ ]:
for i in range(x):
    for j in range(y):
        for k in z:
            check = var[i,j,k].x
            if check == 1:
                kakuro_matrix[i][j] = k

## Solved Kakuro Matrix GUI

In [ ]:
import tkinter as tk

def create_gui(kakuro_matrix):
    root = tk.Tk()
    root.title("Kakuro Matrix Display")
    
    # Setting the grid
    
    canvas = tk.Canvas(root, width=1500, height=1500, bg="black")
    canvas.pack()

    cell_size = 80
    border_width = 2
    text_spacing = 25

    for i, row in enumerate(kakuro_matrix):
        for j, cell in enumerate(row):
            x0 = j * cell_size
            y0 = i * cell_size
            x1 = (j + 1) * cell_size
            y1 = (i + 1) * cell_size

            canvas.create_rectangle(x0, y0, x1, y1, outline="white", width=border_width)

            if not isinstance(cell, tuple):
                # Color cells with the optimal solution in green
                canvas.create_rectangle(x0, y0, x1, y1, outline="black", fill="white")

                # Display the optimal solution in respective cells
                text_x = (x0 + x1) / 2
                text_y = (y0 + y1) / 2
                canvas.create_text(text_x, text_y, text=str(cell), font=("Helvetica", 12), fill="black")
            else:
                
                # Draw diagonal lines for tuple elements
                canvas.create_line(x0, y0, x1, y1, fill="white", width=border_width)

                # Display the clues
                text_x1 = (x0 + x1) / 2 - text_spacing
                text_y1 = (y0 + y1) / 2
                text_x2 = (x0 + x1) / 2 + text_spacing
                text_y2 = (y0 + y1) / 2
                
                
                if cell[0] is None:
                    canvas.create_text(text_x1, text_y1, text='', font=("Helvetica", 12), fill="white")
                else:
                    canvas.create_text(text_x1, text_y1, text=str(cell[0]), font=("Helvetica", 12), fill="white")
                    
                if cell[1] is None:
                    canvas.create_text(text_x2, text_y2, text='', font=("Helvetica", 12), fill="white")
                else:
                    canvas.create_text(text_x2, text_y2, text=str(cell[1]), font=("Helvetica", 12), fill="white")
                

    root.mainloop()

# Create the GUI
create_gui(kakuro_matrix)